Fuente original: https://www.nyc.gov/site/tlc/about/tlc-trip-record-data.page\
En Kaggle: https://www.kaggle.com/datasets/shuhengmo/uber-nyc-forhire-vehicles-trip-data-2021

## Ejemplo prueba VTC NCY 2021

In [1]:
import pandas as pd

In [4]:
path = '../Data/uber-nyc-forhire-vehicles-trip-data-2021/versions/1/'
file = 'fhvhv_tripdata_2021-01.csv'

In [5]:
trips = pd.read_csv(path + file)

In [7]:
trips.isna().sum()

hvfhs_license_num           0
dispatching_base_num        0
originating_base_num    27162
request_datetime            0
on_scene_datetime       27064
pickup_datetime             0
dropoff_datetime            0
PULocationID                0
DOLocationID                0
trip_miles                  0
trip_time                   0
base_passenger_fare         0
tolls                       0
bcf                         0
sales_tax                   0
congestion_surcharge        0
airport_fee             99953
tips                        0
driver_pay                  0
shared_request_flag         0
shared_match_flag           0
access_a_ride_flag          0
wav_request_flag            0
wav_match_flag              0
dtype: int64

In [8]:
trips.drop(columns=['airport_fee'], inplace=True)

In [ ]:
datetime_columns = ['request_datetime', 'on_scene_datetime', 'pickup_datetime', 'dropoff_datetime']
new_trips.drop(columns=datetime_columns, inplace=True)

In [18]:
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestRegressor
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OrdinalEncoder

# Define the columns
object_columns = new_trips.select_dtypes(include=['object']).columns.tolist()

object_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('encoder', OrdinalEncoder())
])

# Combine transformers into a preprocessor
preprocessor = ColumnTransformer(
    transformers=[
        ('object', object_transformer, object_columns)
    ],
    remainder='passthrough'
)


In [19]:
from sklearn.model_selection import train_test_split

# Separate features and target variable
X = new_trips.drop(columns=['driver_pay'])
y = new_trips['driver_pay']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(80000, 18) (20000, 18) (80000,) (20000,)


### Regresor

In [23]:

# Create the pipeline
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', RandomForestRegressor(random_state=13431, n_estimators=10))
])

# Fit the pipeline
pipeline.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('object',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('encoder',
                                                                   OrdinalEncoder())]),
                                                  ['hvfhs_license_num',
                                                   'dispatching_base_num',
                                                   'originating_base_num',
                                                   'shared_request_flag',
                                                   'shared_match_flag',
                                                   'access_a_ride_flag',
                                                   'wav_request_flag',
                                                   'wav_match_flag'])])),
                ('regressor',
                 RandomForestRegressor(n_estimators=10, random_state=13431))])

In [24]:
from sklearn.metrics import r2_score, mean_absolute_error

# Predict on the training data
y_train_pred = pipeline.predict(X_train)

# Evaluate r2_score and mean_absolute_error
r2 = r2_score(y_train, y_train_pred)
mae = mean_absolute_error(y_train, y_train_pred)

print(f'R2 Score on training data: {r2}')
print(f'Mean Absolute Error on training data: {mae}')

R2 Score on training data: 0.9811655021990764
Mean Absolute Error on training data: 0.7421199375


In [25]:
# Predict on the test data
y_test_pred = pipeline.predict(X_test)

# Evaluate r2_score and mean_absolute_error
r2_test = r2_score(y_test, y_test_pred)
mae_test = mean_absolute_error(y_test, y_test_pred)

print(f'R2 Score on test data: {r2_test}')
print(f'Mean Absolute Error on test data: {mae_test}')

R2 Score on test data: 0.8956724178476981
Mean Absolute Error on test data: 1.8789109


### Clustering

In [ ]:
from sklearn.preprocessing import StandardScaler

# Step 1: Create a copy of the trips dataframe
trips_copy = trips.copy()

# Step 2: Convert object type columns using OrdinalEncoder
object_columns = trips_copy.select_dtypes(include=['object']).columns.tolist()
encoder = OrdinalEncoder()
trips_copy[object_columns] = encoder.fit_transform(trips_copy[object_columns])

# Step 3: Drop the datetime variables
datetime_columns = ['request_datetime', 'on_scene_datetime', 'pickup_datetime', 'dropoff_datetime']
trips_copy.drop(columns=datetime_columns, inplace=True)

# Step 4: Apply StandardScaler
scaler = StandardScaler()
trips_scaled = scaler.fit_transform(trips_copy)

# Convert the scaled data back to a DataFrame
trips_scaled_df = pd.DataFrame(trips_scaled, columns=trips_copy.columns)

   hvfhs_license_num  dispatching_base_num  originating_base_num  \
0          -0.607397             -0.795056             -1.371359   
1          -0.607397             -0.795056             -1.371359   
2          -0.607397             -0.688674             -1.242408   
3          -0.607397             -0.688674             -1.242408   
4          -0.607397             -0.688674             -1.242408   

   PULocationID  DOLocationID  trip_miles  trip_time  base_passenger_fare  \
0      1.253846      0.387860    0.113862   0.096006             0.269233   
1      0.250706      0.400381   -0.228483   0.995936            -0.024927   
2      1.292428      0.087344   -0.258252  -0.049081            -0.348352   
3      0.122098      0.099865   -0.847256  -1.362706            -0.809101   
4      0.134959     -0.714033    0.951652   0.693999             0.631679   

     tolls       bcf  sales_tax  congestion_surcharge      tips  driver_pay  \
0 -0.22757  0.175771   0.315483              1.69

In [31]:
from sklearn.impute import SimpleImputer

# Create an imputer object with mean strategy
imputer = SimpleImputer(strategy='mean')

# Apply the imputer to the dataframe
trips_scaled_df_imputed = pd.DataFrame(imputer.fit_transform(trips_scaled_df), columns=trips_scaled_df.columns)

# Display the first few rows of the imputed dataframe
print(trips_scaled_df_imputed.head())

   hvfhs_license_num  dispatching_base_num  originating_base_num  \
0          -0.607397             -0.795056             -1.371359   
1          -0.607397             -0.795056             -1.371359   
2          -0.607397             -0.688674             -1.242408   
3          -0.607397             -0.688674             -1.242408   
4          -0.607397             -0.688674             -1.242408   

   PULocationID  DOLocationID  trip_miles  trip_time  base_passenger_fare  \
0      1.253846      0.387860    0.113862   0.096006             0.269233   
1      0.250706      0.400381   -0.228483   0.995936            -0.024927   
2      1.292428      0.087344   -0.258252  -0.049081            -0.348352   
3      0.122098      0.099865   -0.847256  -1.362706            -0.809101   
4      0.134959     -0.714033    0.951652   0.693999             0.631679   

     tolls       bcf  sales_tax  congestion_surcharge      tips  driver_pay  \
0 -0.22757  0.175771   0.315483              1.69

In [ ]:
from sklearn.cluster import KMeans

# Apply KMeans
kmeans = KMeans(n_clusters=8, random_state=42)
kmeans.fit(trips_scaled_df_imputed)

# Add the cluster labels to the dataframe
trips_scaled_df_imputed['cluster'] = kmeans.labels_

/home/gonzalopc/.local/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


   hvfhs_license_num  dispatching_base_num  originating_base_num  \
0          -0.607397             -0.795056             -1.371359   
1          -0.607397             -0.795056             -1.371359   
2          -0.607397             -0.688674             -1.242408   
3          -0.607397             -0.688674             -1.242408   
4          -0.607397             -0.688674             -1.242408   

   PULocationID  DOLocationID  trip_miles  trip_time  base_passenger_fare  \
0      1.253846      0.387860    0.113862   0.096006             0.269233   
1      0.250706      0.400381   -0.228483   0.995936            -0.024927   
2      1.292428      0.087344   -0.258252  -0.049081            -0.348352   
3      0.122098      0.099865   -0.847256  -1.362706            -0.809101   
4      0.134959     -0.714033    0.951652   0.693999             0.631679   

     tolls       bcf  sales_tax  congestion_surcharge      tips  driver_pay  \
0 -0.22757  0.175771   0.315483              1.69

In [35]:
trips_scaled_df_imputed['cluster'].value_counts()

cluster
0    32710
2    23618
4    22041
1    15462
5     3695
7     2387
3       60
6       27
Name: count, dtype: int64